<a href="https://colab.research.google.com/github/aleks-haksly/ClickHouse/blob/main/practice/part_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
!pip install clickhouse_driver --quiet
from clickhouse_driver import Client
import sqlite3
from google.colab import userdata
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
import matplotlib.pyplot as plt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 935.1/935.1 kB 7.3 MB/s eta 0:00:00


In [2]:
# connection requisites
host = 'clickhouse.lab.karpov.courses'
dbase = 'default'
port = 9000
user = 'student'
password = userdata.get('ch2020[student]')

In [3]:
client = Client(host=host, port=port, user=user,
                password=password, database=dbase)

In [91]:
# Скачаем таблицу целиком
sql = f"""
SELECT
  InvoiceNo,
  StockCode,
  Description,
  Quantity,
  toString(InvoiceDate, 'Europe/London') AS InvoiceDate,
  toString(UnitPrice) AS UnitPrice,
  CustomerID,
  Country
  FROM retail
"""
result, columns = client.execute(sql, with_column_types=True)
df = pd.DataFrame(result, columns=[tuple[0] for tuple in columns])

In [63]:
# Сохраним таблицу в локальную базу sqlite3 retail
connection = sqlite3.connect(r'./Data/retail')
df.to_sql('retail', connection, index=False, if_exists='replace')

406829

In [66]:
# функия для работы с sqlite3 базой retail
def select(sql):
    return pd.read_sql(sql, connection)

### Задание 7.1
Для начала посмотрите, в каких странах имеется наибольшее число уникальных пользователей. Постройте барчарт, где:
*	по оси x – страна
*	по оси y – число пользователей

In [79]:
sql = """
SELECT Country, count(DISTINCT CustomerID) as 'Users count'
FROM retail
GROUP BY Country
"""
df = select(sql)

In [88]:
fig = px.bar(df, x="Country", y="Users count", log_y=True)
fig.update_layout(
    autosize=True,
    xaxis={'categoryorder':'max descending'},
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0,
        pad=0
    ))
fig.show()

### Задание 7.2
Следующая задача – посмотреть на динамику изменения числа активных пользователей в месяц (MAU). В каких месяцах число активных пользователей во Франции было больше, чем в Германии?

In [117]:
sql = """
SELECT
  Country,
  DATE(InvoiceDate, 'start of month')  as date,
  count(DISTINCT CustomerID) as 'uniq users'
FROM
  retail
WHERE
  Country in ('Germany', 'France')
GROUP BY
  Country,
  date
ORDER BY
  date, Country
"""
df = select(sql)

In [118]:
px.line(df, x='date', y='uniq users', color='Country')